In [0]:
#!/usr/bin/env python
# Databricks / PySpark + Delta Lake
# pip install sodapy

from sodapy import Socrata
from delta.tables import DeltaTable
from pyspark.sql import functions as F
import json, os


In [0]:
### ─────────────────────────────────────────────
### 1. Parámetros y utilidades
### ─────────────────────────────────────────────
TOKEN = dbutils.widgets.get("token_app")
DATASET_ID     = dbutils.widgets.get("codigo_dataset")                
DELTA_TABLE    = "main.diplomado_datos.ids_contratos_procesos"
BATCH_SIZE     = 50_000                                                # máximo seguro para SODA 2.0
CTRL_PATH      = "/dbfs/FileStore/tmp/secop_offset.json"               # para reanudar

client = Socrata("www.datos.gov.co", TOKEN, timeout=60)

In [0]:
import time

mode_write = "overwrite"
reintentos = 5

for entidad in lista_entidades:
  intentos = 0
  while intentos < reintentos:
    try:
      query="""
      SELECT * WHERE nit_de_la_entidad = '{0}'
      """.format(entidad.nit_de_la_entidad)
      print(query)
      results_entidad=client.get(codigo_dataset,query=query)
      df_total=spark.createDataFrame(results_entidad)
      df_total.write \
        .format("delta") \
        .mode(mode_write) \
        .option("overwriteSchema", "true") \
        .saveAsTable("diplomado_datos.secop_bronze")
      mode_write = "append"
      break
    except Exception as e:
      intentos += 1
      time.sleep(10)
      print("Error: {0}".format(e))


print("Se cargaron todos los contratos")

In [0]:
df_secop_id=spark.table("diplomado_datos.ids_contratos_procesos")
df_secop_id.count()

In [0]:
# Importar las funciones necesarias de PySpark
from pyspark.sql.functions import sha2, concat_ws, col

# 1. Cargar la tabla correcta desde el catálogo a un DataFrame
# Se asume que esta es la tabla que contiene las columnas que mencionaste.
df_secop_id = spark.table("diplomado_datos.ids_contratos_procesos")

# 2. Definir la lista corregida de columnas para el identificador único
columnas_para_hash = [
    "numero_del_contrato",
    "numero_de_proceso",
    "nit_de_la_entidad",
    "documento_proveedor",
    "estado_del_proceso"
]